In [ ]:
import xopr

import holoviews as hv
import xarray as xr
import hvplot
import hvplot.xarray
import hvplot.pandas
import geoviews.feature as gf
import cartopy.crs as ccrs
import rioxarray
from tqdm import tqdm

In [ ]:
opr = xopr.OPRConnection()

In [ ]:
epsg_3031 = ccrs.Stereographic(central_latitude=-90, true_scale_latitude=-71)
coastline = gf.coastline.options(scale='50m').opts(projection=epsg_3031)
velocity = rioxarray.open_rasterio(
    "https://its-live-data.s3.amazonaws.com/velocity_mosaic/v2/static/cog/ITS_LIVE_velocity_120m_RGI19A_0000_v02_v.tif",
    chunks='auto', overview_level=4, cache=False
).squeeze().drop_vars(['spatial_ref', 'band']).rename('velocity (m/year)')
velocity_map = velocity.hvplot.image(x='x', y='y', cmap='gray_r').opts(clim=(0,1000))

In [ ]:
region = xopr.geometry.get_antarctic_regions(name=["Vincennes_Bay", "Underwood"], merge_regions=True, simplify_tolerance=100)
region_projected = xopr.geometry.project_geojson(region, source_crs='EPSG:4326', target_crs="EPSG:3031")

region_hv = hv.Polygons([region_projected]).opts(
    color='green',
    line_color='black',
    fill_alpha=0.3)

(velocity_map * coastline * region_hv).opts(aspect='equal')

In [ ]:
gdf = opr.query_frames(geometry=region).to_crs('EPSG:3031')
print(f"Found {len(gdf)} radar frames in the selected region.")

In [ ]:
radar_frames_hv = gdf.hvplot(by='collection', hover_cols=['id'])
(velocity_map * coastline * region_hv * radar_frames_hv).opts(aspect='equal', legend_position='top_left')

In [ ]:
layer_ds_list = []

with tqdm(gdf.iterrows(), total=len(gdf)) as t:
    for id, frame in t:
        t.set_description(f"{id}")
        layers = opr.get_layers(frame)
        if 2 not in layers:
            continue # No bed picks available
        layer_wgs84 = xopr.radar_util.layer_twtt_to_range(layers[2], layers[1], vertical_coordinate='wgs84').rename({'lat': 'Latitude', 'lon': 'Longitude'})
        layer_wgs84 = xopr.geometry.project_dataset(layer_wgs84, target_crs='EPSG:3031')
        layer_wgs84 = layer_wgs84.dropna('slow_time', subset=['wgs84'])
        layer_wgs84['source'] = id
        layer_ds_list.append(layer_wgs84)

In [ ]:
bed_merged = xr.concat(layer_ds_list, dim='slow_time')

In [ ]:
bed_hv = bed_merged.hvplot.scatter(x='x', y='y', c='wgs84')
(velocity_map * coastline * region_hv * radar_frames_hv * bed_hv).opts(aspect='equal', legend_position='top_left')